In [1]:
import ollama 
import os
import pandas as pd

In [13]:
model_name = 'evallm:v2'
number_of_repets = 3

In [9]:
powerful_df = pd.read_csv('csv/powerful_model.csv')
powerful_df = powerful_df.drop(columns=['Unnamed: 0'])
powerful_df.head(3)

,model_id,filepath,decoded,references,expert_coherence_score,turker_coherence_score,expert_relevance_score,turker_relevance_score,expert_consistency_score,turker_consistency_score,expert_fluency_score,turker_fluency_score
0,M22,cnndm/dailymail/stories/b5925c1a02fa9a8f6b1d5a...,A video of a crocodile swimming through a publ...,"[""A local bike shop owner's dog was snatched b...",5.000000,4.0,4.333333,4.0,5.0,4.0,5.0,3.8
1,M22,cnndm/dailymail/stories/207df192edc1836250b69d...,Finnish researchers compared weight and height...,"[""Researchers claimed antibiotics could be co...",4.000000,4.0,5.000000,4.2,5.0,4.4,5.0,4.2
2,M22,cnndm/dailymail/stories/6372eff07fd3d6f578f90d...,"Dr Adrian Quarterman , of Dundee University , ...",['Dr Adrian Quarterman has suggested mirrored\...,3.666667,3.6,4.333333,3.8,5.0,3.8,5.0,3.4


In [8]:
client = ollama.Client()
client

In [14]:
deneme_df = powerful_df.head(5)

In [15]:
results = pd.DataFrame(columns=['index', 'coherence', 'fluency', 'relevance', 'consistency'])
results

,index,coherence,fluency,relevance,consistency


In [ ]:
for row in deneme_df.iterrows():
    index, data = row
    file = data['filepath']
    summary = data['decoded']

    print(f'Processing summary {index}...')

    with open(os.path.join('datasets', file), 'r') as f:
        text = f.read()

    query = 'Text:\n' + text + 'Summary:\n' + summary

    result = []
    for i in range(number_of_repets):
        print(f'Repetition {i+1}...')
        response = client.generate(model_name, query)
        response_str = response.response

        coherence = response_str.split('Coherence: ')[1][0]
        fluency = response_str.split('Fluency: ')[1][0]
        relevance = response_str.split('Relevance: ')[1][0]
        consistency = response_str.split('Consistency: ')[1][0]

        result.append({
            'coherence': coherence,
            'fluency': fluency,
            'relevance': relevance,
            'consistency': consistency
        })
    print('Repets are done')
    results_repets = pd.DataFrame(result)
    results_repets = results_repets.astype(float)
    results_repets = results_repets.mean()
    results_repets['index'] = index

    results = results.append(results_repets, ignore_index=True)

Processing summary 0...
Repetition 1...


IndexError: list index out of range